In [10]:
import numpy as np
import pandas as pd
import math
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multinomial
from typing import List
import seaborn as sns

## profit function ##

def profit_t(E,P,vc,fc,pi_tax,c_tax):
    pi = ((E*(P-vc-c_tax))-fc)*(1-pi_tax)
    return pi

## define State class

class State: 
    def __init__(self,mean,eta,lambda_switch,sigma):
        self.m = mean
        self.e = eta 
        self.l = lambda_switch
        self.s = sigma

## state characterisation
        
state_1 = State(75,0.44,0.26,0.23)
state_2 = State(30,1.05,0.28,0.44)

## lattice parameters ##

p_step = 5
s0_step = 5000000

p_1 = 10
p_max = 250

s0_1 = 50000000
s0_max = 500000000

## lattice construction ##

P_grid = list(range(p_1,p_max,p_step))
s0_grid = list(range(s0_1,s0_max,s0_step))

## finite difference parameter definition ##

## parameter defn for P_i in i = 2,...,i_max-1 ##

def difference(P,p_step,sigma,eta,mean):
    a_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    b_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    if {a_i_central and b_i_central >= 0}:
        return [a_i_central,b_i_central]
    elif b_i_forward >= 0:
        return [a_i_forward,b_i_forward]
    else: 
        return [a_i_backward,b_i_backward]
    
## parameter defn for P_i in i = 1 ##
    
def difference_min(P,p_step,eta):
    a_i = 0
    b_i = eta*(mean-P)/(p_step)
    return [a_i,b_i]

## parameter defn for P_i in i = i_max ##

def difference_max(P,p_step,eta):
    a_i = eta*(mean - P)/(p_step)
    b_i = 0
    return [a_i,b_i]

def Profit(P,s_k):
    for i in range(n):
        if s_k > E:
            pi = profit_t(E,P,vc,fc,pi_tax,c_tax)
            return pi 
        elif s_k > 0:
            pi = profit_t(s_k,P,vc,fc,pi_tax,c_tax)
            return pi
        else:
            pi = 0
            return pi

In [11]:
## characterise project ##

P = 60
n = 30
r = 0.05 
E = 300000
vc = 20
fc = 200000
pi_tax = 0.2
c_tax = 5

## create reserves vector ##

s_k = []


## create empty profit and valuation vectors ##

pi_p = []
A = []
A.append(0)

In [13]:
## v_i_k defines iterative matrix of extraction option value for lattice nodes (P_i,S_k) 

v_i_k = np.zeros((len(P_grid),len(s0_grid)))

[[-3760000. -3760000. -3760000. ... -3760000. -3760000. -3760000.]
 [-2560000. -2560000. -2560000. ... -2560000. -2560000. -2560000.]
 [-1360000. -1360000. -1360000. ... -1360000. -1360000. -1360000.]
 ...
 [50240000. 50240000. 50240000. ... 50240000. 50240000. 50240000.]
 [51440000. 51440000. 51440000. ... 51440000. 51440000. 51440000.]
 [52640000. 52640000. 52640000. ... 52640000. 52640000. 52640000.]]


In [21]:
for t in range(n):   
    for i in range(len(P_grid)):
        for k in range(len(s0_grid)):
            if t*E <= s0_grid[k]:
                v_i_k[i,k] = v_i_k[i,k] + profit_t(E,P,vc,fc,pi_tax,c_tax)
            elif {t*E - s0_grid[k] > 0}:
                v_i_k[i,k] = v_i_k[i,k] + profit_t(t*E - s0_grid[k],P,vc,fc,pi_tax,c_tax)
            else:
                v_i_k[i,k] = v_i_k[i,k] + 0

In [22]:
print(v_i_k)

[[-9.49600e+07 -9.49600e+07 -9.49600e+07 ... -9.49600e+07 -9.49600e+07
  -9.49600e+07]
 [ 1.42400e+07  1.42400e+07  1.42400e+07 ...  1.42400e+07  1.42400e+07
   1.42400e+07]
 [ 1.23440e+08  1.23440e+08  1.23440e+08 ...  1.23440e+08  1.23440e+08
   1.23440e+08]
 ...
 [ 4.81904e+09  4.81904e+09  4.81904e+09 ...  4.81904e+09  4.81904e+09
   4.81904e+09]
 [ 4.92824e+09  4.92824e+09  4.92824e+09 ...  4.92824e+09  4.92824e+09
   4.92824e+09]
 [ 5.03744e+09  5.03744e+09  5.03744e+09 ...  5.03744e+09  5.03744e+09
   5.03744e+09]]
